In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from datetime import datetime

#collecting primary articles
stories = []
def getTheGoodStuff(newsstories):
    global stories
    global i
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url,
            "sort_rank" : i}
        stories.append(d)
        for related in data.find_all(class_="related-item"):
            related_htmlatag = related.find("a")
            related_headline = related_htmlatag.getText()
            related_url = related_htmlatag.get("href")
            r = {"headline" : related_headline,
                 "url" : related_url,
                 "sort_rank" : i+1}
            stories.append(r)
            
def scrapeWebsites():
    global stories
    global i
    
    # Getting stories from Fox News.
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    for i in range(0, 20):
        foundstories = soup.find_all("article", class_="article story-" + str(i))
        getTheGoodStuff(foundstories)
    
scrapeWebsites()
articles = pd.DataFrame(stories)
articles.drop_duplicates(inplace=True)
articles['date'] = datetime.today().strftime('%Y-%m-%d')
articles['year'] = datetime.today().year
articles['month'] = datetime.today().month
articles['day'] = datetime.today().day
articles['hour'] = datetime.today().hour
articles['minute'] = datetime.today().minute

fox_domains = []
for web in articles.url:
    domain_items = web.split('//')[-1].split('/')[0].strip('www.').split('.')
    url_item = web.split('https://www.foxnews.com/')[-1].split('/')[0]
    if domain_items[0] in ['video','foxbusiness','nation']:
        fox_domains.append(domain_items[0])
    else:
        fox_domains.append(url_item)
articles['article_type'] = fox_domains

#collecting Opinion articles
sidebar_stories = []

def opinion_getTheGoodStuff(newsstories):
    global sidebar_stories
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url}
        sidebar_stories.append(d)

def opinion_scrapeWebsites():
    global sidebar_stories
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    foundstories = soup.find(class_="collection collection-opinion has-load-more js-opinion").find_all("article")
    for article in foundstories:
        opinion_getTheGoodStuff(foundstories)
        
opinion_scrapeWebsites()
sidebar_articles = pd.DataFrame(sidebar_stories)
sidebar_articles.drop_duplicates(inplace=True)
sort_ranks = range(sidebar_articles.shape[0]+1)
del sort_ranks[0]
sidebar_articles["sort_rank"] = sort_ranks
sidebar_articles['date'] = datetime.today().strftime('%Y-%m-%d')
sidebar_articles['year'] = datetime.today().year
sidebar_articles['month'] = datetime.today().month
sidebar_articles['day'] = datetime.today().day
sidebar_articles['hour'] = datetime.today().hour
sidebar_articles['minute'] = datetime.today().minute

sidebar_articles['article_type'] = "opinion"

all_articles = pd.concat([articles,sidebar_articles],sort=False)

#harvesting article text
from selenium import webdriver
from time import sleep

article_text = []
driver = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

for url in all_articles.url:
    driver.get(url)
    sleep(1) #wait 1 second
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    article_text.append(soup.find_all('p'))

driver.close()    
all_articles['article_text'] = article_text

foxnews_articles = pd.read_csv('foxnews.csv')
print "foxnews articles current shape:"
print foxnews_articles.shape[0]
print ""
conc = pd.concat([foxnews_articles,all_articles],sort=False)
conc.reset_index(inplace=True)
conc.drop(['index','Unnamed: 0'],axis=1,inplace=True)

conc.to_csv('foxnews.csv', encoding = 'utf-8')
print "foxnews new shape:"
print conc.shape[0]